In [ ]:
# Initialize notebook
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

df0 = pd.read_csv('../../assets/data/monsters.csv')
df0 = df0.astype({'Book': 'category', 'Type': 'category', 'Size': 'category'})
df0['All Conditions'] = df0['Blinded'] + df0['Charmed'] + df0['Deafened'] + df0['Exhaustion'] + df0['Frightened'] + df0['Grappled'] + df0['Incapacitated'] + df0['Invisible'] + df0['Paralyzed'] + df0['Petrified'] + df0['Poisoned'] + df0['Prone'] + df0['Restrained'] + df0['Stunned'] + df0['Unconscious']
df0.drop_duplicates(inplace=True)
print('Includes {} monsters in database'.format(len(df0.index.to_list())))

CONDITIONS = ['Blinded','Charmed','Deafened','Exhaustion','Frightened',
    'Grappled','Incapacitated','Invisible','Paralyzed','Petrified',
    'Poisoned','Prone','Restrained','Stunned','Unconscious']
CONDITION_COLORS = ['gold', 'red', 'green', 'orange', 'teal',
    'purple', 'navy', 'maroon', 'olivedrab', 'orangered',
    'blue', 'darkviolet', 'dodgerblue', 'brown', 'darkgreen']

Includes 2117 monsters in database


In [ ]:
# Fig. 1: Plots the average probability that a monster is immune to each condition
#df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1/8, 30)]
df1 = df0[df0['CR'].between(1/8, 30)]

conditions = CONDITIONS.copy()

# create custom text
total_monsters = df1.shape[0]
total_immune = [int(df1[c].sum().tolist()) for c in conditions]
pct_immune = [100*df1[c].mean().tolist() for c in conditions]
custom_text = [f'{p:.0f}% ({i} out of {total_monsters})' for p, i in zip(pct_immune, total_immune)]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            range=[-0.5, len(conditions)-0.5],
            automargin=True,
        ),
        yaxis=dict(
            title_text='percent immune',
            automargin=True,
            range=[-1, 35],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=2.5),
        )
    )
)

fig.add_trace(go.Scatter(
    x=conditions,
    y=pct_immune,
    mode='markers', 
    marker=dict(size=10, line=dict(width=1, color='Black')),
    text=custom_text,
    unselected=dict(marker=dict(opacity=0.0)),
    hovertemplate=
        '<b>Condition:</b> %{x}<br>'+
        '<b>Immune:</b> %{text}'+
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-condition-immunity-probability-large', style='aspect-ratio:600/450')
    tfb.save_fig_html(fig, format='small', name=f'./fig-condition-immunity-probability-small', style='aspect-ratio:600/500')

In [5]:
# Fig. 2: Calculates the probability of a monster for each being immune to each condition.
#df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1/8, 30)]
df1 = df0[df0['CR'].between(1/8, 30)]

# get the total number of monsters from each type
dfG = df1.groupby(['Type'], observed=False).count().reset_index()
type_totals = dfG['Book'].to_list()
print('Total number of monsters:', np.sum(type_totals))

# get the number of monsters immune to each condition for each type
dfG = df1[['Type'] + CONDITIONS].groupby(['Type'], observed=False).sum().reset_index()

# construct custom text for each cell
custom_texts = []
for c in CONDITIONS:
    type_immune = dfG[c].to_list()
    custom_texts.append([f'{i:.0f} out of {t}' for i, t in zip(type_immune, type_totals)])

# get percent of each type immune to each condition
dfG = df1[['Type'] + CONDITIONS].groupby(['Type'], observed=False).mean().reset_index()
z = []
conditions = CONDITIONS.copy()
for condition in conditions:
    row = dfG[condition].tolist()
    row = [int(100*v) for v in row]
    z.append(row)

x = [x.lower() for x in dfG['Type'].tolist()]
y = [x.lower() for x in conditions]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        font=dict(size=12),
        xaxis=dict(
            title_text='',
            automargin=True,
            autorange=True,
            showgrid=False,
        ),
        yaxis=dict(
            title_text='',
            automargin=True,
            autorange=True,
            showgrid=False,
        ),
    )
)


fig.add_trace(go.Heatmap(
    z=z, x=x, y=y,
    zmin=0, zmax=120,
    hoverongaps=False,
    colorscale='blues',
    text=custom_texts,
    texttemplate='%{z:.0f}',
    hovertemplate='<b>Type:</b> %{x}<br>'+
                  '<b>Condition:</b> %{y}<br>'+
                  '<b>Immune:</b> %{z:.0f}% (%{text})'+
                  '<extra></extra>'
))
fig.update_traces(showscale=False)

# show figure
fig.update_layout(width=1000, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-condition-immunity-probability-heat-map-large', style='width:1000px; min-width:50%; max-width:100%; max-height:500px; min-height:300px;')
    tfb.save_fig_html(fig, format='small', name=f'./fig-condition-immunity-probability-heat-map-small', style='width:1000px; min-width:50%; max-width:100%; max-height:500px; min-height:300px;')

Total number of monsters: 2117


In [4]:
# Fig. 3: Plots the percent of monsters immune to each condition as a function of their CR.
conditions = CONDITIONS.copy()
colors = CONDITION_COLORS.copy()
i = conditions.index('Invisible')
conditions.pop(i); colors.pop(i)
i = conditions.index('Incapacitated')
conditions.pop(i); colors.pop(i)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='challenge rating',
            range=[0.0, 30.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='percent immune',
            range=[-5, 105],
            tick0=0, dtick=20,
            minor=dict(tick0=0, dtick=10),
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',  y=1.04,
            xanchor='left',    x=0.01,
            font_size=12,
            #xanchor='left', yanchor='top',
            #x=0.01, y=0.99,
            #orientation='v',
            tracegroupgap=0,
        ),
    )
)

#df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1/8, 30)]
df1 = df0[df0['CR'].between(1/8, 30)]
dfG = df1[['CR'] + conditions].groupby('CR').mean().reset_index()
for condition, color in zip(conditions, colors):
    # generate custom text
    cr_totals = df1[['CR','Book']].groupby('CR').count()['Book'].to_list()
    condition_totals = [int(x) for x in df1[['CR',condition]].groupby('CR').sum()[condition].to_list()]
    custom_text = [f'({i} out of {t})' for i, t in zip(condition_totals, cr_totals)]

    # plot data
    fig.add_trace(go.Scatter(
        x=dfG['CR'],
        y=100*dfG[condition],
        mode='markers', 
        #line=dict(color=color, dash='solid'),
        #marker=dict(size=12, line=dict(width=1, color='Black')),
        marker=dict(size=4, color=color),
        line_shape='spline',
        unselected=dict(marker=dict(opacity=0.0)),
        showlegend=True,
        name=condition,
        legendgroup=condition,
        text=custom_text,
        hovertemplate=
            '<b>CR:</b> %{x}<br>'+
            '<b>Condition:</b> ' + condition + '<br>'+
            '<b>Immune:</b> %{y:.0f}% %{text}'+
            '<extra></extra>'
    ))

    # add fit line
    coefs = np.polyfit(dfG['CR'], 100*dfG[condition], 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=dfG['CR'],
        y=poly(dfG['CR']),
        mode='lines', 
        line=dict(color=color, dash='solid'),
        unselected=dict(marker=dict(opacity=0.0)),
        showlegend=False,
        legendgroup=condition,
        legendgrouptitle_text='fit',
        hoverinfo='skip',
    ))

# show figure
fig.update_layout(width=600, height=550)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-condition-immunity-prob-vs-cr-large', style='aspect-ratio:600/550', marker_size=4)
    tfb.save_fig_html(fig, legend_font_size=8, format='small', name=f'./fig-condition-immunity-prob-vs-cr-small', style='aspect-ratio:600/600', marker_size=3)

In [7]:
# Plots average number of conditional immunities for monsters at each CR.
conditions = ['All Conditions']
colors = ['black']

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='challenge rating',
            range=[0.0, 30.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='condition immunities',
            range=[-0.05, 6],
            tick0=0, dtick=1.0,
            minor=dict(tick0=0, dtick=0.25),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            orientation='v',
        )
    )
)

#df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1/8, 30)]
df1 = df0[df0['CR'].between(1/8, 30)]
dfG = df1[['CR'] + conditions].groupby('CR').mean()
for condition, color in zip(conditions, colors):
    dfR = dfG[condition].rolling(1, center=True).mean()
    fig.add_trace(go.Scatter(
        x=dfR.index.to_series(),
        y=dfR,
        mode='lines+markers', 
        line=dict(color=color, dash='solid'),
        unselected=dict(marker=dict(opacity=0.0)),
        name=condition,
        hovertemplate=
            'CR %{x}<br>'+
            'immunities %{y:.2f}'+
            '<extra></extra>'
    ))


# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    """fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-condition-immunity-prob-vs-cr-large', style='aspect-ratio:600/500')
    tfb.save_fig_html(fig, format='small', name=f'./fig-condition-immunity-prob-vs-cr-small', style='aspect-ratio:600/500')"""
    pass

In [10]:
# Plots the average number of condition immunities for monsters of each type.
condition = 'All Conditions'

#df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1/8, 30)]
df1 = df0[df0['CR'].between(1/8, 30)]
dfG = df1[['Type',condition]].groupby(['Type'], observed=False).mean().reset_index()

fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='',
            automargin=True,
            range=[-0.5, 14-0.5],
        ),
        yaxis=dict(
            title_text=f'condition immunities',
            range=[0.0, 6.0],
            tick0=0, dtick=1.0,
            minor=dict(tick0=0, dtick=0.5),
        )
    )
)

fig.add_trace(go.Histogram(
    histfunc='avg', 
    x=dfG['Type'], 
    y=dfG[condition], 
    name=condition,
    hovertemplate=
        '<b>%{x}:</b> %{y:.2f}<br>'+
        '<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    """fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-condition-immunity-prob-vs-cr-large', style='aspect-ratio:600/550')
    tfb.save_fig_html(fig, format='small', name=f'./fig-condition-immunity-prob-vs-cr-small', style='aspect-ratio:600/550')"""
    pass

In [ ]:
# Runs a linear regression fit for immunity to one condition based on monster CR and type.
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

yCol = 'Poisoned'
xCols = ['CR','Type']
aCols = [yCol] + xCols
df1 = df0[aCols]
X = pd.get_dummies(data=df1[xCols], drop_first=False)
Y = df1[yCol]

model = LinearRegression()
model.fit(X, Y)

coeff_parameter = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])

# generate report
X_Sm = sm.add_constant(X)
ls = sm.OLS(Y, X_Sm).fit()
print(ls.summary())